# Hyperparameter Search
Use Bayesian optimization to find (learning-rate and momentum). The advantage of bayesian optimization compared to grid/random search is that those methods are less sample efficient compared to bayesian optimization.

#### References
* https://en.wikipedia.org/wiki/Bayesian_optimization
* https://towardsdatascience.com/a-conceptual-explanation-of-bayesian-model-based-hyperparameter-optimization-for-machine-learning-b8172278050f
* https://ax.dev
* https://ax.dev/tutorials/tune_cnn.html
* https://botorch.org
* https://papers.nips.cc/paper/4443-algorithms-for-hyper-parameter-optimization.pdf
* http://krasserm.github.io/2018/03/21/bayesian-optimization/

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from typing import Dict, Tuple
import torch.utils.data as utils
from torchvision.utils import make_grid
from torch.utils.tensorboard import SummaryWriter
from torch.optim import lr_scheduler

from sklearn.model_selection import train_test_split
# Library for augmentations on batch of numpy/tensors
from imgaug import augmenters as iaa

import sat_utils
import seg_loss
import seg_metrics
import seg_models

from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax.service.managed_loop import optimize
from ax.utils.notebook.plotting import render, init_notebook_plotting
from ax.utils.tutorials.cnn_utils import load_mnist, train, evaluate

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = 'cpu'
print('Device:', device)
num_gpu = torch.cuda.device_count()
print('Number of GPUs Available:', num_gpu)

lr=1e-6 #0.001 0.0001-(Good with Dice, 0.007 training)
l2_norm=0.0000001
gamma=0.1
batch_size = 64 #32 #20
num_epochs = 30
step_size = 200

writer = None

Device: cuda:0
Number of GPUs Available: 8


### Load Data from pickle (Bad not scalable) and create data loader

In [2]:
X = sat_utils.read_pickle_data('./data/input.pickle')
Y = sat_utils.read_pickle_data('./data/label.pickle')

# Convert dictionaries to numpy array
X = np.stack([sat_utils.get_rgb(x) for x in X.values()])
Y = np.stack([(x/255.0) for x in Y.values()])

# Split train/validation
X_t, X_v, Y_t, Y_v = train_test_split(X, Y, test_size=1/10, random_state=42)
print('X_t:', X_t.shape)
print('Y_t:', Y_t.shape)
print('X_v:', X_v.shape)
print('Y_v:', Y_v.shape)

# Changes on label for Cross-Entropy (3 classes all mixed on the same image, N,W,H)
# Changes on label for BCEWithLogitsLoss (3 classes on 3 Channels, N,C,W,H)
tensor_x_t = torch.Tensor(X_t)
tensor_y_t = torch.Tensor(Y_t)
tensor_x_v = torch.Tensor(X_v)
tensor_y_v = torch.Tensor(Y_v)

# Define some augmentations
seq_augm = iaa.Sequential([
    iaa.Fliplr(0.5), # horizontally flip 50% of the images
    iaa.Flipud(0.5), # vertically flip 50% of the images
    #iaa.Affine(rotate=(-10, 10)), # Rotate the images
])

dataset_train = utils.TensorDataset(tensor_x_t,tensor_y_t)
dataset_val = utils.TensorDataset(tensor_x_v,tensor_y_v)
dataloader_train = utils.DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
dataloader_val = utils.DataLoader(dataset_val, batch_size=batch_size, shuffle=False)

num_classes = tensor_y_t.shape[1]

X_t: (2038, 3, 76, 76)
Y_t: (2038, 3, 76, 76)
X_v: (227, 3, 76, 76)
Y_v: (227, 3, 76, 76)


#### Define Train Function
Train the model for one epoch given:
* Train loader
* Parameters

In [3]:
def train(train_loader: DataLoader, parameters: Dict[str, float], dtype: torch.dtype, device: torch.device) -> nn.Module:
    # Start Tensorboard interface
    global writer
    writer = SummaryWriter()
    
    # Initialize Model and optimizer
    model = seg_models.AtrousSeg(num_classes=num_classes, num_channels=tensor_x_t.shape[1])
    
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    
    model.to(device)
    
    # Define loss and optimizer
    #optimizer = optim.SGD(model.parameters(), lr=parameters.get("lr", 0.001), momentum=parameters.get("momentum", 0.9))
    base_lr = parameters.get("lr", 0.001)
    optimizer = torch.optim.Adam(model.parameters(), lr=base_lr, weight_decay=l2_norm)
    criterion = nn.BCEWithLogitsLoss()
    
    # Decrease learning rate if some metric doesnt change for "patience" epochs
    scheduler_plateau = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=4, verbose=False)
    
    base_loss_weight = parameters.get("loss_weight", 1.0)
    writer.add_scalar('loss_weight/', base_loss_weight, 0)
    writer.add_scalar('base_lr/', base_lr, 0)
    
    model.train()
    
    # Train Network
    iteration_count = 0
    for epoch in range(num_epochs):
        running_loss = 0.0
        for (imgs, labels) in train_loader:

            # Do augmentations the augmentation library expect numpy arrays on format Batch x Row x Cols x Channels
            imgs_aug, labels_aug = seq_augm(images=np.moveaxis(imgs.numpy(),1,3), heatmaps=np.moveaxis(labels.numpy(),1,3))

            # Move axis back and convert back to tensor
            imgs = torch.from_numpy(np.moveaxis(imgs_aug,3,1))
            labels = torch.from_numpy(np.moveaxis(labels_aug,3,1))

            # Send inputs/labels to GPU                
            labels = labels.to(device)
            imgs = imgs.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(imgs)
            dice_value  = seg_loss.dice_loss(outputs, labels)
            loss = (base_loss_weight*criterion(outputs, labels)) + dice_value
            running_loss += loss.item()
            
            loss.backward()
            optimizer.step()
            writer.add_scalar('loss/', loss.item(), iteration_count)
            writer.add_scalar('dice_loss/', dice_value.item(), iteration_count)
            iteration_count+=1  
        
        # Finished epoch
        # Get current learning rate (To display on Tensorboard)
        for param_group in optimizer.param_groups:
            curr_learning_rate = param_group['lr']
            writer.add_scalar('learning_rate/', curr_learning_rate, epoch)
        
        # Step learning rate scheduler
        scheduler_plateau.step(running_loss)
    return model

#### Define Evaluation Function
Evaluate the model given:
* Some Model
* Data Loader

In [ ]:
def evaluate(model: nn.Module, data_loader: DataLoader, dtype: torch.dtype, device: torch.device) -> float:    
    global writer 
    model.eval()
    iteration_count_val = 0
    total = 0
    iou_val_batch = 0
    with torch.no_grad():
        for (imgs, labels) in data_loader:
            # Send inputs/labels to GPU                
            labels = labels.to(device)
            imgs = imgs.to(device)                
            outputs = model(imgs)
            iou_value = seg_metrics.iou(outputs, labels)
            iou_val_batch += iou_value.item()
            total += labels.size(0)
    
    iou_val = iou_val_batch / total
    writer.add_scalar('iou_end/', iou_val, 0)

    return iou_val

#### Define our objective Function
This is the function we want to optimize given it's parameters. We want to find the parameters that will maximize accuracy on the validation set.

In [ ]:
def objective_func(parameterization):
    model = train(train_loader=dataloader_train, parameters=parameterization, dtype=torch.float, device=device)
    return evaluate(
        model=model,
        data_loader=dataloader_val,
        dtype=torch.float,
        device=device,
    )

#### Run Bayesian Optimizer
Search for best learning rate and momentum hyperparameters.

In [ ]:
best_parameters, values, experiment, model = optimize(
    parameters=[
        {"name": "lr", "type": "range", "bounds": [1e-5, 1e-1], "log_scale": True},
        {"name": "loss_weight", "type": "range", "bounds": [0.0, 1.0]},
    ],
    evaluation_function=objective_func,
    objective_name='IoU',
)

[INFO 05-30 01:08:40] ax.service.utils.dispatch: Using Bayesian Optimization generation strategy. Iterations after 5 will take longer to generate due to model-fitting.
[INFO 05-30 01:08:40] ax.service.managed_loop: Started full optimization with 20 steps.
[INFO 05-30 01:08:40] ax.service.managed_loop: Running optimization trial 1...
[INFO 05-30 01:33:52] ax.service.managed_loop: Running optimization trial 2...
[INFO 05-30 01:58:41] ax.service.managed_loop: Running optimization trial 3...
[INFO 05-30 02:23:21] ax.service.managed_loop: Running optimization trial 4...
[INFO 05-30 02:47:58] ax.service.managed_loop: Running optimization trial 5...
[INFO 05-30 03:12:39] ax.service.managed_loop: Running optimization trial 6...
[INFO 05-30 03:38:08] ax.service.managed_loop: Running optimization trial 7...
[INFO 05-30 04:03:12] ax.service.managed_loop: Running optimization trial 8...
[INFO 05-30 04:28:51] ax.service.managed_loop: Running optimization trial 9...
[INFO 05-30 04:54:25] ax.service.

#### Show Results

In [8]:
print('Best parameters:',best_parameters)
print('IoU:', values)
#render(plot_contour(model=model, param_x='lr', param_y='momentum', metric_name='accuracy'))

Best parameters: {'lr': 0.00010242674403542709, 'loss_weight': 0.43974536824818683}
IoU: ({'IoU': 0.006645133133671554}, {'IoU': {'IoU': 2.9800379620516516e-14}})
